# Сбор данных с сайта РИА Новости

In [ ]:
%%capture
!pip install selenium

In [ ]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
SLEEP = 2
DEPTH = 20 #число страниц для скрапинга
BASE_URL = "https://ria.ru/"
TOPICS = ["economy", "society", "incidents", "science", "culture"]

In [ ]:
@dataclass
class Article:
    id: str = None
    url: str = None
    title: str = None
    subtitle: str = None
    content: str = None
    datetime: str = None

In [ ]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

Функция для скачивания страниц

In [ ]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(SLEEP)

            driver.execute_script(
                "document.getElementsByClassName('list-more')[0].click()"
            )
            time.sleep(1)

            for i in tqdm(range(DEPTH), leave=False):
                try:
                    driver.execute_script(
                        f"window.scrollTo(0, document.body.scrollHeight - 1200)"
                    )
                    time.sleep(1)
                except:
                    pass

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            scope = soup.find(
                "div", {"class": "list", "itemtype": "http://schema.org/ItemList"}
            )
            items += scope.find_all("div", {"class": "list-item"})

            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))
        except:
            pass

    return items, topics_order

Функция для парсинга скачанных страниц.

In [ ]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article url
    article.url = page.find("a", {"class": "list-item__image"})["href"]

    # article id
    s = re.findall(r"\d+.html", article.url)[0]
    article.id = s[: s.find(".")]

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article source
    source = article.url[8 : article.url.find(".")]

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find(
        "div",
        {
            "class": lambda x: x and (x.find(f"article m-article m-{source}") > -1),
            "data-article-id": article.id,
        },
    )

    if not obj:
        obj = soup.find(
            "div",
            {
                "class": lambda x: x and (x.find(f"article m-video m-{source}") > -1),
                "data-article-id": article.id,
            },
        )

    # process article title
    title = obj.find("div", {"class": "article__title"})
    title_2 = obj.find("h1", {"class": "article__title"})

    if title:
        article.title = title.text
    else:
        article.title = title_2.text if title_2 else ""

    # article subtitle
    subtitle = obj.find("h1", {"class": "article__second-title"})
    article.subtitle = subtitle.text if subtitle else ""

    # article content
    article.content = obj.find(
        "div", {"class": "article__body js-mediator-article mia-analytics"}
    ).text

    # article datetime
    article.datetime = obj.find("div", {"class": "article__info-date"}).find("a").text

    # article number of views
    # article.views = int(obj.find('span', {'class': 'statistic__item m-views'}).text)

    return article

Скачайте новости.

In [ ]:
pages, topics_order = get_pages()

100%|██████████| 5/5 [02:40<00:00, 32.15s/it]


Сохраним результат в файл.

In [ ]:
with open("ria_pages.txt", "w", encoding="utf-8") as f:
    for page in pages:
        f.write("%s\n" % page)

## Вопрос 1

Сколько страниц `pages` мы спарсили?

In [ ]:
len(pages)

1808

Распарсим каждую страницу и достанем из нее необходимые атрибуты:

* в цикле примените функцию `parse_page` к каждой странице и сохраните результат в объект `res`

* затем добавьте `res` в список `data`


Для скорости распарсим только первые 100 страниц.

In [ ]:
data, topics_order_fixed = [], []

for num, page in enumerate(tqdm(pages[:100])):
    try:
        res = parse_page(page)
        data.append(res)

        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

100%|██████████| 100/100 [09:57<00:00,  5.97s/it]


## Вопрос 2

Преобразуйте в датафрейм полученные данные.

Сколько столбцов в таблице `df`?

In [ ]:
df = pd.DataFrame(data=data)

df.head()

In [ ]:
num_columns = df.shape[1]
num_columns

6

Добавьте в таблицу `df` столбец `topic`, содержащий список `topics_order_fixed`

In [ ]:
df["topic"] = topics_order_fixed

In [ ]:
df.head(3)

,id,url,title,subtitle,content,datetime,topic
0,1949766100,https://ria.ru/20240531/kurs-valyut-1949766100...,"Курс доллара на Московской бирже вырос до 90,1...","Курс доллара на Мосбирже вырос до 90,1 рубля, ...","МОСКВА, 31 мая - РИА Новости. Курс рубля по ит...",19:14 31.05.2024,economy
1,1949758154,https://ria.ru/20240531/putin-1949758154.html,Путин поручил разработать план развития индуст...,Путин поручил разработать план развития индуст...,"МОСКВА, 31 мая - РИА Новости. Президент России...",18:35 31.05.2024,economy
2,1949719963,https://realty.ria.ru/20240531/minfin-19497199...,Минфин оценил доходы регионов от повышения нал...,Минфин: повышение налогов на дорогостоящее иму...,"МОСКВА, 31 мая - РИА Новости. Повышенное налог...",16:28 31.05.2024,economy


## Вопрос 3

Какой `topic` имеет новость из первой строки таблицы?

In [ ]:
first_row_topic = df.loc[0, 'topic']
first_row_topic

'economy'

Сохраните скачанные данные в файл.

In [ ]:
df.to_pickle("df_ria.p", compression="gzip")

## Вопрос 4

Есть ли пропуски в каких-либо столбцах в таблице `df`?

In [ ]:
missing = df.isnull().sum()
missing

id          0
url         0
title       0
subtitle    0
content     0
datetime    0
topic       0
dtype: int64